<a href="https://colab.research.google.com/github/smayd01/ml_sirius/blob/main/Homework_GradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Градиентный бустинг на решающих деревьях

## Как правильно перебирать параметры

В этом ноутбуке мы будем заниматься поиском оптимальных параметров для градиентного бустинга, перебирая задания по заданной сетке. В этом задании от Вас не будет требоваться найти самые лучшие параметры, но всё равно важно правильно составлять сетку для перебора. Для этого нужно понимать суть параметров и их смысл.



**learning_rate** -- темп обучения нашего метода. Для этого метода сетка перебора должна быть логарифмической, т.е. перебирать порядковые значения (к примеру, [1e-3, 1e-2, 1e-1, 1]). В большинстве случаев достаточно перебрать значения от 1e-5 до 1.<br />
**max_depth** -- максимальная глубина деревьев в ансамбле. Вообще говоря, эта величина зависит от числа признаков, но обычно лучше растить небольшие деревья. К примеру, библиотека CatBoost, которую мы будем исследовать сегодня, рекомендует перебирать значения до 10 (и уточняется, что обычно оптимальная глубина лежит от 6 до 10).<br />
**n_estimators** -- количество деревьев в ансамбле. Обычно стоит перебирать с каким-то крупным шагом (можно по логарифмической сетке). Здесь важно найти баланс между производительностью, временем обучения и качеством. Обычно нескольких тысяч деревьев бывает достаточно.<br />

Учтите, что в реальных задачах необходимо следить за тем, что оптимальные значения параметров не попадают на границы интервалов, т.е. что вы нашли хотя бы локальный минимум. Если Вы перебрали значения параметра от 1 до 10 и оказалось, что 10 - оптимальное значение, значит следует перебрать и бОльшие числа, чтобы убедиться, что качество не улучшается дальше (или по крайней мере убедиться, что рост качества сильно замедляется и на сильное улучшения рассчитывать не стоит.


## Подготовка датасета

Все библиотеки, используемые сегодня, мы будем проверять на одних и тех же параметрах: n_estimators=1000, max_depth=5, learning_rate=0.1. Таким образом мы устанавливаем, соответственно, число деревьев в ансамбле равным 1000, ограничиваем максимальную глубину деревьев 5 и устанавливаем темп обучения равным 0.1.

In [ ]:
!pip install catboost
%matplotlib inline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_absolute_error, make_scorer

from hyperopt import hp, tpe, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll import scope

from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

from catboost import CatBoostRegressor

import matplotlib.pyplot as plt

import pandas as pd

import numpy as np

import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 8.0 MB/s eta 0:00:00


In [ ]:
test_parameters = {"n_estimators": 1000, "max_depth": 5, "learning_rate":0.1}

df = pd.read_csv('/content/dataframe_YesIndex_YesHeader_C.csv', index_col=0)
df.head()

,Engine Capacity,Cylinders,Drive Type,Fuel Tank Capacity,Fuel Economy,Fuel Type,Horsepower,Torque,Transmission,Top Speed,...,Acceleration,Length,Width,Height,Wheelbase,Trunk Capacity,name,price,currency,Country
0,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,Mitsubishi Attrage 2021 1.2 GLX (Base),34099.0,0,0
1,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,Mitsubishi Attrage 2021 1.2 GLX (Base),34099.0,0,0
2,1.4,4,0,45.0,6.3,0,75,118.0,1,156,...,16.0,3.864,1.716,1.721,2.513,2800.0,Fiat Fiorino 2021 1.4L Standard,41250.0,0,0
3,1.6,4,0,50.0,6.4,0,102,145.0,0,180,...,11.0,4.354,1.994,1.529,2.635,510.0,Renault Symbol 2021 1.6L PE,44930.0,0,0
4,1.5,4,0,48.0,5.8,0,112,150.0,0,170,...,10.9,4.314,1.809,1.624,2.585,448.0,MG ZS 2021 1.5L STD,57787.0,0,0


В глаза бросаются две проблемы: у нас имеется информация о производителе автомобиля, скрытая в текстовой переменной модели автомобиля (нет отдельного признака). Также имеются некоторые категориальные переменные.

**Данные**: датасет со стоимостью поддержанных автомобилей  
**Цели**: В данном задании следует выполнить следующие пункты (выполнять можно в любом порядке)
1. Изучить датасет, проверить наличие пропусков. При необходимости заменить их на среднее значение признака.
3. Добавить столбец brand с информацией о производителе автомобиля (для простоты можно взять первое слово в названии модели). Столбец name удалить из датасета
4. Решить, какие признаки Вы хотите сделать категориальными. Конвертировать выбранные категориальные столбцы в тип category.
5. Создать датасет А с категориальными признаками в виде категорий. Для этого необходимо создать вектор целевых значений (столбец цен автомобилей) и матрицу признаков с категориальными переменными в виде категорий (получается путем удаления только целевой переменной из матрицы с данными). Дополнительно стоит создать список с названиями и индексами столбцов категориальных переменных (поможет в будущем).
6. Создать датасет B, с удаленными категориальными признаками.
7. Создать датасет C с категориальными признаками в виде one-hot encoding. Для этого необходимо создать вектор целевых значений (столбец цен автомобилей), удалить из матрицы признаков столбец с целевыми переменными и все категориальными переменные, а затем добавить новые признаки, соответствующие one-hot encoding категориальных переменных (здесь вам поможет функция `pd.get_dummies`).
8. Разбить датасеты на тренировочное и тестовое множества, используя `train_test_split(X, y, test_size=0.25, random_state=0)`

In [ ]:
# Проверка наличия пропусков
print("Пропуски в данных:")
print(df.isnull().sum())

Пропуски в данных:
Engine Capacity       0
Cylinders             0
Drive Type            0
Fuel Tank Capacity    0
Fuel Economy          0
Fuel Type             0
Horsepower            0
Torque                0
Transmission          0
Top Speed             0
Seating Capacity      0
Acceleration          0
Length                0
Width                 0
Height                0
Wheelbase             0
Trunk Capacity        0
name                  0
price                 0
currency              0
Country               0
dtype: int64


In [ ]:
# Добавить столбец brand с информацией о производителе автомобиля
df['Brand'] = df['name'].str.split().str[0]
df.drop(columns=['name'], inplace=True)
df.head()

,Engine Capacity,Cylinders,Drive Type,Fuel Tank Capacity,Fuel Economy,Fuel Type,Horsepower,Torque,Transmission,Top Speed,...,Acceleration,Length,Width,Height,Wheelbase,Trunk Capacity,price,currency,Country,Brand
0,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,34099.0,0,0,Mitsubishi
1,1.2,3,0,42.0,4.9,0,76,100.0,0,170,...,14.0,4.245,1.670,1.515,2.550,450.0,34099.0,0,0,Mitsubishi
2,1.4,4,0,45.0,6.3,0,75,118.0,1,156,...,16.0,3.864,1.716,1.721,2.513,2800.0,41250.0,0,0,Fiat
3,1.6,4,0,50.0,6.4,0,102,145.0,0,180,...,11.0,4.354,1.994,1.529,2.635,510.0,44930.0,0,0,Renault
4,1.5,4,0,48.0,5.8,0,112,150.0,0,170,...,10.9,4.314,1.809,1.624,2.585,448.0,57787.0,0,0,MG


In [ ]:
categorical_columns = ['Drive Type', 'Fuel Type', 'Transmission', 'currency', 'Country', 'Brand', 'Cylinders']

for col in categorical_columns:
    df[col] = df[col].astype('category')

In [ ]:
# Создание датасета A с категориальными признаками в виде категорий
A = df.copy()

y = A['price']

A = A.drop(columns=['price'])

categorical_columns = A.columns.tolist()
categorical_column_indices = [A.columns.get_loc(col) for col in categorical_columns]



In [ ]:
B = df.copy()
B = B.drop(columns=['price', 'Drive Type', 'Fuel Type', 'Transmission', 'currency', 'Country', 'Brand', 'Cylinders'])

In [ ]:
C = df.copy()
C_drop = C.drop(columns=['price', 'Drive Type', 'Fuel Type', 'Transmission', 'currency', 'Country', 'Brand'])
categorical_features = pd.get_dummies(df[['Drive Type', 'Fuel Type', 'Transmission', 'currency', 'Country', 'Brand', 'Cylinders']])

C = pd.concat([C_drop, categorical_features], axis=1)

In [ ]:
# # Удаляем столбец с целевыми переменными (цены автомобилей)
# category_columns=['Drive Type', 'Fuel Type', 'Transmission', 'currency', 'Country', 'Brand']
# target_column = 'price'
# target_values = df[target_column]

# # Создаем датасет А с категориальными переменными в виде категорий
# df_A = df.drop(target_column, axis=1)


# # 5.
# df_B = df.drop(category_columns, axis=1)


# #6.
# # Удаляем категориальные переменные
# df1 = df_A.drop(category_columns, axis=1)

# # Применяем one-hot encoding к категориальным переменным и объединяем с остальными признаками
# df_C = pd.concat([df1, pd.get_dummies(df[category_columns])], axis=1)

# A = df_A
# B = df_B
# C = df_C

# y = target_values

In [ ]:
X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(A, y, test_size=0.25, random_state=0)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(B, y, test_size=0.25, random_state=0)
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(C, y, test_size=0.25, random_state=0)

**Задания**:
1. Обучите любую понравившуюся вам модель градиентного бустинга (CatBoost, XGBoost, LightGBM) для предсказания стоимости автомобиля на всех построенных датасетах (A, B и C)
2. Подберите оптимальный набор параметров модели с помощью библиотеки hyperopt

In [ ]:
from lightgbm import LGBMRegressor

# Обучение и оценка модели на датасете A
# model_A = LGBMRegressor()
model_A = XGBRegressor(enable_categorical=True)
model_A.fit(X_train_A, y_train_A)
score_A = model_A.score(X_test_A, y_test_A)
print("Score on dataset A:", score_A)

# Обучение и оценка модели на датасете B
# model_B = LGBMRegressor()
model_B = XGBRegressor(enable_categorical=True)
model_B.fit(X_train_B, y_train_B)
score_B = model_B.score(X_test_B, y_test_B)
print("Score on dataset B:", score_B)

# Обучение и оценка модели на датасете C
# model_C = LGBMRegressor()
model_C = XGBRegressor(enable_categorical=True)
model_C.fit(X_train_C, y_train_C)
score_C = model_C.score(X_test_C, y_test_C)
print("Score on dataset C:", score_C)

Score on dataset A: 0.970150195569011
Score on dataset B: 0.25532860672846336
Score on dataset C: 0.9603481469700808


In [ ]:
from hyperopt import hp, fmin, tpe

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [1, 2, 6, 7, 8, 9, 10]),
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-2, 1e-1, 1, 3, 5]),
    'n_estimators': hp.choice('n_estimators', [1, 5, 10, 100, 1000, 10000]),
}

# Функция для оптимизации
def objective(params):
    model = XGBRegressor(enable_categorical=True)
    model.fit(X_train_A, y_train_A)
    score = model.score(X_test_A, y_test_A)
    return -score  # Минимизация

# Поиск оптимальных параметров
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)

print("Best parameters:", best_params)

100%|██████████| 50/50 [00:31<00:00,  1.60trial/s, best loss: -0.970150195569011]
Best parameters: {'learning_rate': 4, 'max_depth': 3, 'n_estimators': 4}


In [ ]:
from hyperopt import hp, fmin, tpe

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [1, 2, 6, 7, 8, 9, 10]),
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-2, 1e-1, 1, 3, 5]),
    'n_estimators': hp.choice('n_estimators', [1, 5, 10, 100, 1000, 10000]),
}

# Функция для оптимизации
def objective(params):
    model = XGBRegressor(enable_categorical=True)
    model.fit(X_train_B, y_train_B)
    score = model.score(X_test_B, y_test_B)
    return -score  # Минимизация

# Поиск оптимальных параметров
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)

print("Best parameters:", best_params)

100%|██████████| 50/50 [00:13<00:00,  3.59trial/s, best loss: -0.25532860672846336]
Best parameters: {'learning_rate': 5, 'max_depth': 2, 'n_estimators': 4}


In [ ]:
from hyperopt import hp, fmin, tpe

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [1, 2, 6, 7, 8, 9, 10]),
    'learning_rate': hp.choice('learning_rate', [1e-3, 1e-2, 1e-1, 1, 3, 5]),
    'n_estimators': hp.choice('n_estimators', [1, 5, 10, 100, 1000, 10000]),
}

# Функция для оптимизации
def objective(params):
    model = XGBRegressor(enable_categorical=True)
    model.fit(X_train_C, y_train_C)
    score = model.score(X_test_C, y_test_C)
    return -score  # Минимизация

# Поиск оптимальных параметров
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50)

print("Best parameters:", best_params)

100%|██████████| 50/50 [00:23<00:00,  2.17trial/s, best loss: -0.9603481469700808]
Best parameters: {'learning_rate': 0, 'max_depth': 5, 'n_estimators': 5}
